In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import re


# Spécification du type de données pour la colonne 'SIREN_SIRET'
df = pd.read_csv('data/ecoles.csv', sep=';', dtype={'SIREN_SIRET': str})
pd.set_option('display.max_columns', 69)
pd.set_option('display.max_rows', 500)

In [2]:
 # list(df.columns)

###  Filtrage des champs  

In [3]:
df.drop(columns=[
         'Identifiant_de_l_etablissement',
         'Type_etablissement',
         'Adresse_2',
         'Code postal',
         'Code_commune',
         'Nom_commune',
         'Code_academie',
         'Code_region',
         'Ecole_maternelle',
         'Voie_generale',
         'Voie_technologique',
         'Voie_professionnelle',
         'Fax',
         'Web',
         'Restauration',
         'Hebergement',
         'ULIS',
         'Apprentissage',
         'Segpa',
         'Section_arts',
         'Section_cinema',
         'Section_theatre',
         'Section_sport',
         'Section_internationale',
         'Section_europeenne',
         'Lycee_Agricole',
         'Lycee_militaire',
         'Lycee_des_metiers',
         'Post_BAC',
         'Appartenance_Education_Prioritaire',
         'GRETA',
         'SIREN_SIRET',
         'Fiche_onisep',
         'position',
         'Type_contrat_prive',
         'Libelle_departement',
         'Libelle_academie',
         'Libelle_region',
         'coordonnee_X',
         'coordonnee_Y',
         'epsg',
         'nom_circonscription',
         'latitude',
         'longitude',
         'precision_localisation',
         'date_ouverture',
         'date_maj_ligne',
         'etat',
         'ministere_tutelle',
         'etablissement_multi_lignes',
         'rpi_concentre',
         'rpi_disperse',
         'code_nature',
         'libelle_nature',
         'Code_type_contrat_prive',
         'PIAL',
         'etablissement_mere',
         'type_rattachement_etablissement_mere',
         'code_bassin_formation',
         'libelle_bassin_formation'], inplace=True)

### Filtrage écoles élémentaires uniquement 

In [4]:
df['Ecole_elementaire'].value_counts()

1.0    36966
0.0    13665
Name: Ecole_elementaire, dtype: int64

In [5]:
filt = (df['Ecole_elementaire'] == 1.0)

In [6]:
df = df.loc[filt]

In [7]:
df['Ecole_elementaire'].value_counts()

1.0    36966
Name: Ecole_elementaire, dtype: int64

### Filtrage des adresses mails non valides

In [8]:
df.shape

(36966, 9)

On filtre la forme des mails grâce à une expression régulière et on élimine les ecoles sans adresse mail.

In [9]:
mask = df['Mail'].str.contains(r'[^@]+@[^@]+\.[^@]+') & (df['Mail'] != np.nan)
df = df[mask]
df.shape

(36653, 9)

In [10]:
df['Mail'].str.contains(r'[^@]+@[^@]+\.[^@]+').value_counts()

True    36653
Name: Mail, dtype: int64

Il ne reste bien que des école élémentaires au nombre de 36653 (Publiques + privées), avec des emails valides.

###   Trie par départements 

In [11]:
# Trie par ordre croissant
df = df.sort_values(by='Code_departement')

### Concaténation des adresses et supréssion des colonnes inutiles.


In [12]:
df['Adresse'] = df['Adresse_1'] + ' ' + df['Adresse_3']

On enleve les colonnes inutiles

In [13]:
df = df.drop(columns=['Statut_public_prive', 'Adresse_1', 'Adresse_3', 'Ecole_elementaire', 'Nombre_d_eleves'])

In [14]:
df.shape

(36653, 5)

In [15]:
df.head(1)

,Nom_etablissement,Code_departement,Telephone,Mail,Adresse
40454,ECOLE ELEMENTAIRE PUBLIQUE,01,0472250117,ce.0010501P@ac-lyon.fr,Grande rue 01360 BRESSOLLES


In [16]:
data = df.values.tolist()

On créer une database avec les champs du formulaire utilisateur google contact depuis nos données sous forme de liste data (plus performant de creer une dataFrame depuis une liste que de creer une dataframe vide puis de l'implementer avec les données d'une autre dataframe)

### Faire une df_contact grâce à un csv exporté depuis google_contact pour avoir tous les noms de champs.

In [17]:
df_contact = pd.read_csv('data/contacts.csv')
df_contact.head(0)

,Name,Given Name,Additional Name,Family Name,Yomi Name,Given Name Yomi,Additional Name Yomi,Family Name Yomi,Name Prefix,Name Suffix,Initials,Nickname,Short Name,Maiden Name,Birthday,Gender,Location,Billing Information,Directory Server,Mileage,Occupation,Hobby,Sensitivity,Priority,Subject,Notes,Language,Photo,Group Membership,E-mail 1 - Type,E-mail 1 - Value,Phone 1 - Type,Phone 1 - Value,Address 1 - Type,Address 1 - Formatted,Address 1 - Street,Address 1 - City,Address 1 - PO Box,Address 1 - Region,Address 1 - Postal Code,Address 1 - Country,Address 1 - Extended Address


| df      | df_contact |
| ----------- | ----------- |
| Nom_etablissement      | Name       |
| Adresse  | Address 1 - Formatted        |
| Code_departement   | Group Membership        |
| Telephone   | Phone 1 - Value        |
| Mail   | E-mail 1 - Value        |


In [18]:
df.columns = ['Name', 'Group Membership', 'Phone 1 - Value', 'E-mail 1 - Value', 'Address 1 - Formatted']

### Ajout des colonnes obligatoires du formulaire google_contact à df.

Récupération des colonnes sous forme de listes

In [19]:
contact_col = list(df_contact.columns)
df_col = list(df.columns)

In [20]:
col_a_ajouter = [e for e in contact_col if e not in df_col]

Ajout des colonnes google contact à notre df.

In [21]:
df[col_a_ajouter] = [np.nan for i in range(len(col_a_ajouter))]

In [22]:
df.head(1)

,Name,Group Membership,Phone 1 - Value,E-mail 1 - Value,Address 1 - Formatted,Given Name,Additional Name,Family Name,Yomi Name,Given Name Yomi,Additional Name Yomi,Family Name Yomi,Name Prefix,Name Suffix,Initials,Nickname,Short Name,Maiden Name,Birthday,Gender,Location,Billing Information,Directory Server,Mileage,Occupation,Hobby,Sensitivity,Priority,Subject,Notes,Language,Photo,E-mail 1 - Type,Phone 1 - Type,Address 1 - Type,Address 1 - Street,Address 1 - City,Address 1 - PO Box,Address 1 - Region,Address 1 - Postal Code,Address 1 - Country,Address 1 - Extended Address
40454,ECOLE ELEMENTAIRE PUBLIQUE,01,0472250117,ce.0010501P@ac-lyon.fr,Grande rue 01360 BRESSOLLES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Exportations des écoles par départements en csv


In [23]:
departements = set(df['Group Membership'])

for d in departements:
    filt = df['Group Membership'] == d
    df[filt].to_csv(f'data/ecoles_par_departement/dep_{d}.csv')
    

### Conclusion

Depuis le fichiers des établissements scolaires du ministère, nous avons un fichier csv par département compatible avec l'importation de contact 'google contact'.   

Chaque contact possede le nom de l'établissement son numéro de téléphone son adresse mail et son adresse postale.

# Visualisation


In [24]:
# Spécification du type de données pour la colonne 'SIREN_SIRET'
df_viz = pd.read_csv('https://data.education.gouv.fr/explore/dataset/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B', sep=';', dtype={'SIREN_SIRET': str})
pd.set_option('display.max_columns', 69)
pd.set_option('display.max_rows', 500)

In [25]:
df_viz.columns

Index(['Code établissement', 'Appellation officielle',
       'Dénomination principale', 'Patronyme uai', 'Secteur Public/Privé',
       'Adresse', 'Lieu dit', 'Boite postale', 'Code postal',
       'Localite d'acheminement', 'Commune', 'Coordonnee X', 'Coordonnee Y',
       'EPSG', 'Latitude', 'Longitude', 'Qualité d'appariement',
       'Localisation', 'Code nature', 'Nature', 'Code état établissement',
       'Etat établissement', 'Code département', 'Code région',
       'Code académie', 'Code commune', 'Département', 'Région', 'Académie',
       'Position', 'secteur_prive_code_type_contrat',
       'secteur_prive_libelle_type_contrat', 'code_ministere',
       'libelle_ministere', 'date_ouverture'],
      dtype='object')

In [26]:
df_viz.head(1)

,Code établissement,Appellation officielle,Dénomination principale,Patronyme uai,Secteur Public/Privé,Adresse,Lieu dit,Boite postale,Code postal,Localite d'acheminement,Commune,Coordonnee X,Coordonnee Y,EPSG,Latitude,Longitude,Qualité d'appariement,Localisation,Code nature,Nature,Code état établissement,Etat établissement,Code département,Code région,Code académie,Code commune,Département,Région,Académie,Position,secteur_prive_code_type_contrat,secteur_prive_libelle_type_contrat,code_ministere,libelle_ministere,date_ouverture
0,0040106U,Ecole maternelle,ECOLE MATERNELLE PUBLIQUE,NaN,Public,Place de la Mairie,Le Village,NaN,4200,CHATEAUNEUF VAL ST DONAT,Châteauneuf-Val-Saint-Donat,936096.0,6337116.0,EPSG:2154,44.093362,5.949128,MANUEL,BATIMENT,101,ECOLE MATERNELLE,1.0,OUVERT,04,93,2,04053,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur,Aix-Marseille,"44.0933624599,5.9491279977",99.0,SANS OBJET,6,MINISTERE DE L'EDUCATION NATIONALE,1969-04-24


Il faut filtrer le nombre d'élèves par écoles( si > 2000 ) 

In [27]:
%matplotlib

departements = sorted(departements)
eleves_par_dep_total = []
el_par_dep_primaire = []
el_par_dep_maternelle = []


for d in departements:
    filtre_dep = df_viz['Code_departement'] == d
    filtre_primaire = filtre_dep & df_viz['Ecole_elementaire'] == 1
    
    eleves_par_dep_total.append(df_viz.loc[filtre_dep, 'Nombre_d_eleves'].sum())
    el_par_dep_primaire.append(df_viz.loc[filtre_primaire, 'Nombre_d_eleves'].sum())

plt.bar(departements, eleves_par_dep_total, label='total')
plt.bar(departements, el_par_dep_primaire, label='primaire')


plt.xlabel('Départements')
plt.ylabel('Éleves')
plt.title('Élèves par départements')


plt.gcf().subplots_adjust()

plt.legend()
plt.grid(False)
plt.tight_layout()
# plt.savefig('plot.png')
plt.show()


Using matplotlib backend: TkAgg


KeyError: 'Code_departement'